---
title: QuadGK
venue: Modules
---

## Numerical evaluation of improper integrals

The <b> QuadGK</b> package transforms an infinite interval into a finite one, then aplies the Gauss-Kronrod quadrature. It avoids evaluating the endpoints to prevent singularities: https://juliamath.github.io/QuadGK.jl/stable/quadgk-examples/#quadgk-examples

<b> Example: </b> Integrate the Gauss function $f(x)=e^{-x^2}$ over the real line $(-\infty,\infty)$

In [ ]:
using QuadGK
f(x)=exp(-x^2);
result=quadgk(f,-Inf,Inf)[1]

The exact value for the integral is $\int_{-\infty}^{\infty}f(x) \ dx =\sqrt{\pi}$

In [ ]:
print("The numerical error for the integral is ", result-sqrt(pi))

You can also specify the accuracy of the numerical integral:

In [ ]:
result2=quadgk(f,-Inf,Inf, rtol=1e-3)[1]
print("The numerical error for the less accurate integral is ", result2-sqrt(pi))

<b> Example: </b> Integral containing singularity $\int_0^1 \frac{dx}{\sqrt{x}}=2$

In [ ]:
g(x)=1/sqrt(x);
result=quadgk(g,0,1,rtol=1e-15)[1]
print("The numerical error for the integral is ", result-2)

You can also specify the singularity within the interval, for example in  $\int_{-1}^1 \frac{\sin(x)}{x} dx$

In [ ]:
h(x)=sin(x)/x;
result=quadgk(h,-1,0,1,rtol=1e-15)[1]

# Solutions to ODE and PDE using Improper Integrals

A <b> mass-spring-damper </b> system with displacement $x$, mass $m$, damping constant $d$, and spring constant $k$, while subject to an external force $f$, is governed by the following initial value problem:

\begin{equation}
m\ddot{x}+b\dot{x}+kx=f, \ x(0)=x_0,\  \dot{x}(0)=v_0
\end{equation}



The above equation can be solved with the help of <b> Laplace transforms </b>. Let $X=\mathcal{L}x, F=\mathcal{L}f$ be the Laplace transforms of $x,f$. Note that $X(s),F(s)$ are functions of a complex variable $s=\rho+\sigma \cdot i$. Then for zero initial values $x_0=0,v_0=0$, the solution $x$ can be obtained via inverse Laplace transform as follows:

\begin{equation}
 x(t) = \frac{1}{2\pi} \int_{-\infty}^{\infty} e^{(\rho+\sigma i)t}\cdot \frac{F(\rho+\sigma i)}{(\rho+ \sigma i)^2+(\rho+\sigma i)+1 } \ d\sigma  
\end{equation}

Where $\rho$ is a fixed constant such that all poles of the integrand lie to the left of $\rho$ on the complex plane, i.e. their imaginary parts are less than $\rho$.


<b>Example:</b> Let $m=b=k=1$. The input forcing is unit step $u(t)$ with Laplace transform $U(s)=\frac{1}{s}$  Choose $\rho=2$. Then the solution is given by

\begin{equation}
 x(t) = \frac{1}{2\pi} \int_{-\infty}^{\infty} e^{2t}e^{\sigma i t}\cdot \frac{ d\sigma }{(2+\sigma i)[(2+ \sigma i)^2+(2+\sigma i)+1 ]}  
\end{equation}



In [ ]:
#Setup system parameters

using Plotly, QuadGK
unit_step(x)=0.5*(sign(x)+1)
Laplace_ustep(s)=1/(s)

#Material constants for mass spring damper system
mass=1
damp=1
spring=1


#Laplace transforms of transfer function and inpu forcing
transfer_function(s)=1/(mass*s .^2 +damp*s+spring)
input_f_lapl(s)=Laplace_ustep(s)

fun_lapl(s)=transfer_function.(s).*input_f_lapl.(s)

#rho
margin=2

In [ ]:

#Parameters for exact solution
omega=sqrt(3/4)

zeta1=-0.5
zeta2=-0.5
a=-1
b=-1/sqrt(3)
c=1

#Start computations
t_start=0
t_end=8

time=t_start:0.4:t_end
integrand(s)=exp.((margin+s*im)*time).*fun_lapl(margin+s*im);
fun_solution=(2pi)^(-1)*quadgk(integrand,-Inf,Inf)[1];

#Overdamped solution
#exact=a*exp.(zeta1*time) .+ b*exp.(zeta2*time) .+ c  

#Underdampled solution
exact=a*exp.(zeta1*time).*cos.(omega*time) .+ b*exp.(zeta2*time).*sin.(omega*time) .+ c 


trace1 = scatter(x=time,y=unit_step.(time),mode="markers", name="Input Forcing")
trace2 = scatter(x=time,y=real(fun_solution),mode="markers", name="Numerical solution")
trace3 = scatter(x=time,y=exact,mode="lines", name="Exact solution")
#trace2=scatter(time,fun_solution)
layout = Layout(;title="Forced Mass Spring Damper System ", xaxis_range=[t_start, t_end], xaxis_title="Time",
                     yaxis_range=[0, 1.3],  yaxis_title="Displacement")

plot([trace1, trace2, trace3],layout)

More examples to play around with: http://lampx.tugraz.at/~hadley/physikm/apps/odeanalytic/dharmonic.en.php

# Solution of linear PDE: Heat Equation

The evolution of temperature distribution $T(x,t)$ on an infinitely long thin rod with thermal conductivity $\kappa$ and initial temperature $T(x,t=0)=T_0(x) $ without external input is governed by the    <b> heat equation:</b>

\begin{equation}
\frac{\partial T}{\partial t}=\kappa \frac{\partial^2 T}{\partial x^2}, \ t>0, \ x\in (-\infty,\infty), \ T(x,0)=T_0(x) 
\end{equation}



The above equation can be solved via convolution of the initial distribution $T_0$ with the <b> heat kernel </b> $\Phi(x,t)=\frac{1}{\sqrt{4\pi\kappa t}}\exp(-\frac{x^2}{4\kappa t}) $. The solution can be written as
\begin{equation}
 T(x,t) =  \int_{-\infty}^{\infty} \Phi(x-x',t)T_0(x')\  dx' =  \int_{-\infty}^{\infty} \frac{1}{\sqrt{4\pi\kappa t}}\cdot \exp\left(-\frac{(x-x')^2} {4\kappa t} \right) T_0(x')\  dx'
\end{equation}

<b> Example: </b> Let the initial distribution be a Gaussian pulse centered around the origin $x=0$:

In [ ]:
# Plot initial Gaussian distribution
startx=-5
endx=5
x = LinRange(startx,endx,200)
x0=0


initial_dist(x)= exp(-x.^2)

# trace = scatter(x=cos.(ω1*t),y=sin.(ω2*a0*t),mode="lines", name="Lissajous")
trace = scatter(x=x,y=initial_dist.(x),mode="lines", name="Lissajous")
data = [trace]
layout = Layout(;title="Time = 0", xaxis_range=[startx, endx], xaxis_title="x",
                     yaxis_range=[-1.1, 1.1],  yaxis_title="Temperature ", width=1000,height=500)
pl = plot(data,layout)

Solution to the heat equation on the infinite line interval $[-\infty,\infty]$ with initial temperature distribution $T_0(x)=e^{-x^2}$:

\begin{equation}
 T(x,t) =  \int_{-\infty}^{\infty} \Phi(x-x',t)T_0(x')\  dx' =  \int_{-\infty}^{\infty} \frac{1}{\sqrt{4\pi \kappa t}}\cdot \exp\left(-\frac{(x-y)^2} {4 \kappa t} \right) e^{-y^2}\  dy 
 \end{equation}

In [ ]:
Nt=25
t0 = 0.0001
t1 = 100
k=0.01

heat_kernel(x,t)=1 ./sqrt.(4*pi*k*t).*exp.(-x.^2 ./(4*k.*t))

for time =LinRange(t0,t1,Nt)
  # Setup integrand for each time
    integrand(y)=heat_kernel.(x.-y,time).*initial_dist(y)
    
  #Compute exact solution at each time with integral formula  
    
    temp=quadgk(integrand,-Inf,Inf)[1]
   trace = scatter(x=x,y=temp,mode="lines", name="Lissajous") 
    data = [trace]
    #Animate results
layout = Layout(;title="Time = "*string(time), xaxis_range=[startx, endx], xaxis_title="x",
                     yaxis_range=[-1.1, 1.1],  yaxis_title="Temperature ", width=1000,height=500)
    
    sleep(0.05)
    react!(pl, data, layout)
end

